In [128]:
import pandas as pd
import requests
import json
import os
import openpyxl
import subprocess
import git

In [129]:
#Barrier Token Generator
ps_script_name = 'PSBarrierToken.ps1'
result = subprocess.run(['powershell.exe', './' + ps_script_name], capture_output=True, text=True)
output = result.stdout
# Print the output
print(output)




Environment : Public
TenantId    : 92431631-d203-4da0-b62c-6fd3e495252f
UserName    : akhil.grandhi@dunboxed.com

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Inp4ZWcyV09OcFRrd041R21lWWN1VGR0QzZKMCIsImtpZCI6Inp4ZWcyV09OcFRrd041R21lWWN1VGR0QzZKMCJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvOTI0MzE2MzEtZDIwMy00ZGEwLWI2MmMtNmZkM2U0OTUyNTJmLyIsImlhdCI6MTczMjI1MTk5NiwibmJmIjoxNzMyMjUxOTk2LCJleHAiOjE3MzIyNTczODIsImFjY3QiOjAsImFjciI6IjEiLCJhaW8iOiJBWlFBYS84WUFBQUFaYnRsaXRoU1ljMTZJYnN2TnBZUXN5R05aR1ZPWmp2OXJPYVAxV1dGeDRNR3BDbEROdllPOFJhR0hzaUlmTXpMbUhNQ1JKVDBQb1k1c296QVBQVXI5N09yYS9XaTc4RzlyK2lISWkyakExQVVLc29oVVltSGF2bUdaM3RIazk3QlhxZkN0MnBBRHhKY2VJeVFkdzgyK0xFL1hVRERuK0R5T3BCMDJIRlh4ekg4Z3FTSmt3SWsxZ0JKN2RjbkY1ZisiLCJhbXIiOlsicnNhIiwibWZhIl0sImFwcGlkIjoiMjNkOGY2YmQtMWViMC00Y2MyLWEwOGMtN2JmNTI1YzY3YmNkIiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJjYzY5ODNkOC1iYjVmLTQ5MzQtYjkxYi1iOTVjYjE2ZGUwMTUiLCJmYW1pbHlfbmFtZSI6IkdyYW5kaGkiLCJnaXZ

In [130]:
Token = "Bearer " + output.split(".com")[1][2:-3]
headers={'Authorization': Token,
'Content-Type':'application/json'}
body={
}

In [131]:
#Update the Paths with PROD Name Manually we are hard coding the Workspace Names Example: _Adhoc_Model_Prod
#Update the repo path to pull changes to a a partciular folder
#Use this folder while pulling data from GIT
#Place your clone link in the repo_url
#Errors Array
Errors = []

In [132]:
# Clone the repository if it doesn't exist, or fetch the latest changes if it already exists
repo_path = "C://Users//AkhilGrandhi//Downloads//PBIADO/Testing" # Specify the local path where you want to clone the repository
repo_url = "https://akhilgrandhiTesting@dev.azure.com/akhilgrandhiTesting/Testing/_git/Testing"  # Specify the Git repository URL

try:
    # If the repository already exists, fetch the latest changes
    repo = git.Repo(repo_path)
    origin = repo.remote(name="origin")
    origin.fetch()
    origin.pull()

    print("Successfully pulled the latest changes.")
except git.exc.NoSuchPathError:
    # If the repository doesn't exist, clone it
    repo = git.Repo.clone_from(repo_url, repo_path)

    print("Successfully cloned the repository.")
except git.exc.GitCommandError as e:
    Errors.append(["An error occurred while pulling the latest changes:"+e])
    print(f"An error occurred while pulling the latest changes: {e}")


Successfully pulled the latest changes.


In [133]:
#Get all the dataset details
url="https://api.powerbi.com/v1.0/myorg/groups"
dataset_response = requests.get(url,headers=headers)
if dataset_response.status_code == 200:
    jsonstr = dataset_response.content
    workspaces = json.loads(jsonstr)
    # List to store data source that needs to be ignored as invalid data sources
    workspacesData = workspaces['value']
    workspaceIDs = []
    isReadOnly = []
    isOnDedicatedCapacity=[]
    capacityId = [] 
    defaultDatasetStorageFormat=[]
    workspaceType=[]
    workspaceNames = []

    # requiredWorkspaceNames = ['']
    
    for workspace in workspacesData:
        if('id' in workspace):
            workspaceIDs.append(workspace['id'])
        if('isReadOnly' in workspace):
            isReadOnly.append(workspace['isReadOnly'])
        else:
            isReadOnly.append('NA')
        if('isOnDedicatedCapacity' in workspace):
            isOnDedicatedCapacity.append(workspace['isOnDedicatedCapacity'])
        else:
            isOnDedicatedCapacity.append('NA')
        if( 'capacityId' in workspace):
            capacityId.append(workspace['capacityId'])
        else:
            capacityId.append('NA')
        if( 'name' in workspace):
            workspaceNames.append(workspace['name'])
        else:
            workspaceNames.append('NA')
        if( 'defaultDatasetStorageFormat' in workspace):
            defaultDatasetStorageFormat.append(workspace['defaultDatasetStorageFormat'])
        else:
            defaultDatasetStorageFormat.append('NA')
        if( 'type' in workspace):
            workspaceType.append(workspace['type'])
        else:
            workspaceType.append('NA')

    tempDict = {'WorkspaceID':workspaceIDs,'IsReadOnly':isReadOnly,'IsOnDedicatedCapacity':isOnDedicatedCapacity,'CapacityID':capacityId,
                'DefaultDatasetStorageFormat':defaultDatasetStorageFormat,'WorkspaceType':workspaceType,'WorkspaceName': workspaceNames,}
    df_workspace_names_ids = pd.DataFrame(tempDict)
else:
    Errors.append(["Error at Workspaces_Data",dataset_response])

In [134]:
Errors

[]

In [135]:
#Workspaces data excel creation
df_workspace_names_ids.to_excel("Workspaces_Data.xlsx")

In [136]:
#Stroing All Sematic model details
df_Workspace_Datasets = pd.DataFrame()
WorkspaceID = []
WorkspaceNames = []
DatasetIds = []
DatasetNames = []
DatasetWebUrl = []
ConfiguredBy = []
CreatedDate = []
IsRefreshable = []
queryScaleOutSettings = []
maxReadOnlyReplicas = []
autoSyncReadOnlyReplicas = []


for Id in range(len(workspaceIDs)):
    workspaceUrl = "https://api.powerbi.com/v1.0/myorg/groups/"+workspaceIDs[Id]+"/datasets"
    Workspace_Datasets_response = requests.get(workspaceUrl,headers=headers)
    if Workspace_Datasets_response.status_code == 200:
        DatasetJsonstr = Workspace_Datasets_response.content
        datasets = json.loads(DatasetJsonstr)
        datasetsData = datasets['value']
        for dataset in datasetsData:
            WorkspaceID.append(workspaceIDs[Id])
            WorkspaceNames.append(workspaceNames[Id])
            DatasetIds.append(dataset['id'])
            DatasetNames.append(dataset['name'])
            DatasetWebUrl.append(dataset['webUrl'])
            try:
                ConfiguredBy.append(dataset['configuredBy']) 
            except:
                ConfiguredBy.append('Microsoft')
            CreatedDate.append(dataset['createdDate'])
            IsRefreshable.append(dataset['isRefreshable'])
            if(dataset['queryScaleOutSettings']):
                queryScaleOutSettings.append(True)
                autoSyncReadOnlyReplicas.append(dataset['queryScaleOutSettings']['autoSyncReadOnlyReplicas'])
                maxReadOnlyReplicas.append(dataset['queryScaleOutSettings']['maxReadOnlyReplicas'])
            else:
                queryScaleOutSettings.append(False)
                autoSyncReadOnlyReplicas.append(dataset['queryScaleOutSettings']['autoSyncReadOnlyReplicas'])
                maxReadOnlyReplicas.append(dataset['queryScaleOutSettings']['maxReadOnlyReplicas'])
    else:
        Errors.append(["Error at Workspaces_Datasets in "+workspaceIDs[Id],Workspace_Datasets_response])

            
tempDict = {'WorkspaceID':WorkspaceID,'WorkspaceName': WorkspaceNames,'DatasetName': DatasetNames,
                    'DatasetID':DatasetIds,'DatasetWebUrl':DatasetWebUrl, 'ConfiguredBy':ConfiguredBy,'CreatedDate':CreatedDate, 
                    'IsRefreshable':IsRefreshable,'QueryScaleOutSettings':queryScaleOutSettings,
                    'AutoSyncReadOnlyReplicas':autoSyncReadOnlyReplicas,'MaxReadOnlyReplicas':maxReadOnlyReplicas}
df_Workspace_Datasets = pd.DataFrame(tempDict)

In [137]:
Errors

[]

In [138]:
#Workspace Datasets excel creation
df_Workspace_Datasets.to_excel("Workspaces_Datasets.xlsx")

In [139]:
#Getting Soures of a dataset
DatasourceType = []
Server_Path = []
Database_Kind = []
DatasourceId = []
GatewayId = []
DatasetID = []
WorkspaceID = []


for index, row in df_Workspace_Datasets.iterrows():
    sourceUrl = "https://api.powerbi.com/v1.0/myorg/groups/"+row['WorkspaceID']+"/datasets/"+row['DatasetID']+"/datasources"
    Dataset_SourceDetails_response = requests.get(sourceUrl,headers=headers)
    
    if Dataset_SourceDetails_response.status_code == 200:
        sourceJsonstr = Dataset_SourceDetails_response.content
        source = json.loads(sourceJsonstr)
        sourceData = source['value']
        #print(sourceData)

        DatasetID.append(row['DatasetID'])
        WorkspaceID.append(row['WorkspaceID'])
        try:
            DatasourceType.append(sourceData[0]['datasourceType'])
        except:
            DatasourceType.append("NA")
        #connectionDetails
        try:
            if("path" in sourceData[0]['connectionDetails'] and "kind" in sourceData[0]['connectionDetails']):
                Server_Path.append(sourceData[0]['connectionDetails']['path'])
                Database_Kind.append(sourceData[0]['connectionDetails']['kind'])
            elif("server" in sourceData[0]['connectionDetails'] and "database" in sourceData[0]['connectionDetails']):
                Server_Path.append(sourceData[0]['connectionDetails']['server'])
                Database_Kind.append(sourceData[0]['connectionDetails']['database'])
            elif("url" in sourceData[0]['connectionDetails']):
                Server_Path.append(sourceData[0]['connectionDetails']['url'])
                Database_Kind.append("NA")
            else:
                # print(sourceData[0]['connectionDetails'])
                Server_Path.append(sourceData[0]['connectionDetails'])
                Database_Kind.append("NA")
        except:
            Server_Path.append("NA")
            Database_Kind.append("NA")
        #datasourceId
        try:
            DatasourceId.append(sourceData[0]['datasourceId'])
        except:
            DatasourceId.append("NA")
        #gatewayId
        try:
            GatewayId.append(sourceData[0]['gatewayId'])
        except:
            GatewayId.append("NA")
    else:
        # print(row['DatasetID'],row['DatasetName'])
        Errors.append(["Error at Dataset_SourceDetails in "+row['DatasetID']+" "+ row['DatasetName'],Dataset_SourceDetails_response])


sourceDict = {'WorkspaceID': WorkspaceID, 'DatasetID': DatasetID, 'DatasourceType': DatasourceType, 
            'Server_Path':Server_Path , 'Database_Kind': Database_Kind, 'DatasourceID': DatasourceId, 'GatewayID': GatewayId}
df_sourceDetails = pd.DataFrame(sourceDict)
    

In [140]:
Errors

[['Error at Dataset_SourceDetails in 3563efcb-ffe1-4369-b4bb-467d4d47c48d Feature Usage and Adoption',
  <Response [404]>],
 ['Error at Dataset_SourceDetails in b07832c6-31c9-4153-825e-207685400d5f Purview Hub',
  <Response [404]>]]

In [141]:
#Workspace Datasets excel creation
df_sourceDetails.to_excel("Dataset_SourceDetails.xlsx")

In [142]:
Errors = []

In [143]:
#Datasets Refresh Details
DatasetID = []
Frequency = []
Days = []
Times = []
Enabled = []
LocalTimeZoneId = []
NotifyOption = []


for index, row in df_Workspace_Datasets.iterrows():
    try:
        refreshUrl = "https://api.powerbi.com/v1.0/myorg/datasets/"+row['DatasetID']+"/refreshSchedule"
        refreshUrl_response = requests.get(refreshUrl,headers=headers)
        if refreshUrl_response.status_code == 200:
            refreshJsonstr = requests.get(refreshUrl,headers=headers).content
            refreshData = json.loads(refreshJsonstr)
        else:
            # print("else case:",row['DatasetId'])
            refreshUrl = "https://api.powerbi.com/v1.0/myorg/datasets/"+row['DatasetID']+"/directQueryRefreshSchedule"
            refreshUrl_response = requests.get(refreshUrl,headers=headers)
            if refreshUrl_response.status_code == 200:
                refreshJsonstr = refreshUrl_response.content
                refreshData = json.loads(refreshJsonstr)
            else:
                Errors.append(["Error at Dataset_RefreshDetails in "+row['DatasetID'+" "+row['DatasetName']],refreshUrl_response])
                if('error' in refreshData):
                    print(row['DatasetID'],row['DatasetName'])
        #print(refreshData)
        if('frequency' in refreshData):
            Frequency.append(refreshData['frequency'])
        else:
            Frequency.append("NA")
        if('days' in refreshData):
            Days.append(refreshData['days'])
        else:
            Days.append("NA")
        if('times' in refreshData):
            Times.append(refreshData['times'])
        else:
            Times.append("NA")
        if('localTimeZoneId' in refreshData):
            LocalTimeZoneId.append(refreshData['localTimeZoneId'])
        else:
            LocalTimeZoneId.append("NA")
        if('notifyOption' in refreshData):
            NotifyOption.append(refreshData['notifyOption'])
        else:
            NotifyOption.append("NA")
        DatasetID.append(row['DatasetID'])
    except:
        DatasetID.append(row['DatasetID'])
        Frequency.append("NA")
        Days.append("NA")
        Times.append("NA")
        LocalTimeZoneId.append("NA")
        NotifyOption.append("NA")
        

refreshDic = {'DatasetID': DatasetID, 'Frequency': Frequency, 'Days': Days, 'Times': Times, 
              'LocalTimeZoneId': LocalTimeZoneId, 'NotifyOption': NotifyOption}
df_refreshDetails = pd.DataFrame(refreshDic)

In [144]:
Errors

[]

In [145]:
#Shedule Refreshes of Datasets excel creation
df_refreshDetails.to_excel("Dataset_RefreshDetails.xlsx")

In [146]:
#Getting Users and gropus of a dataset
DatasetUserAccessRight = []
Identifier = []
PrincipalType = []
DatasetID = []
WorkspaceID = []


for index, row in df_Workspace_Datasets.iterrows():
    usersUrl = "https://api.powerbi.com/v1.0/myorg/groups/"+row['WorkspaceID']+"/datasets/"+row['DatasetID']+"/users"
    usersUrl_response = requests.get(usersUrl,headers=headers)
    if usersUrl_response.status_code == 200:
        usersJsonstr = usersUrl_response.content
        users = json.loads(usersJsonstr)
        try:
            usersData = users['value']
            for i in range(len(usersData)):
                DatasetUserAccessRight.append(usersData[i]['datasetUserAccessRight'])
                Identifier.append(usersData[i]['identifier'])
                PrincipalType.append(usersData[i]['principalType'])
                DatasetID.append(row['DatasetID'])
                WorkspaceID.append(row['WorkspaceID'])
        except:
            print(row['DatasetID'])
            DatasetUserAccessRight.append("NA")
            Identifier.append("NA")
            PrincipalType.append("NA")
            DatasetID.append(row['DatasetID'])
            WorkspaceID.append(row['WorkspaceID'])
    else:
        Errors.append(["Error at Dataset_UsersDetails in "+row['DatasetID'],usersUrl_response])

        
usersDict = {'WorkspaceID': WorkspaceID, 'DatasetID': DatasetID, 'DatasetUserAccessRight': DatasetUserAccessRight, 
            'Identifier':Identifier , 'PrincipalType': PrincipalType}
df_usersDetails = pd.DataFrame(usersDict)

In [147]:
Errors

[['Error at Dataset_UsersDetails in 3563efcb-ffe1-4369-b4bb-467d4d47c48d',
  <Response [401]>],
 ['Error at Dataset_UsersDetails in b07832c6-31c9-4153-825e-207685400d5f',
  <Response [401]>]]

In [148]:
#Datasets user details excel creation
df_usersDetails.to_excel("Dataset_UsersDetails.xlsx")

In [149]:
#Pulling Gateway Details
GatewayId = []
GatewayName = []
Type = []
GatewayAnnotation = []


gatewayUrl = "https://api.powerbi.com/v1.0/myorg/gateways"
gatewayUrl_response = requests.get(gatewayUrl,headers=headers)
if gatewayUrl_response.status_code == 200:
    gatewayJsonstr = gatewayUrl_response.content
    gateways = json.loads(gatewayJsonstr)
    gatewayData = gateways['value']
    for gateway in gatewayData:
        GatewayId.append(gateway['id'])
        GatewayName.append(gateway['name'])
        Type.append(gateway['type'])
        GatewayAnnotation.append(gateway['gatewayAnnotation'])
else:
    Errors.append(["Error at GatewayDetails",gatewayUrl_response])

    
gatewayDict = {'GatewayID':GatewayId, 'GatewayName':GatewayName, 'Type':Type, 'GatewayAnnotation':GatewayAnnotation}
df_gatewayDetails = pd.DataFrame(gatewayDict)

In [151]:
Errors

[['Error at Dataset_UsersDetails in 3563efcb-ffe1-4369-b4bb-467d4d47c48d',
  <Response [401]>],
 ['Error at Dataset_UsersDetails in b07832c6-31c9-4153-825e-207685400d5f',
  <Response [401]>]]

In [150]:
#Gateway details excel creation
df_gatewayDetails.to_excel("GatewayDetails.xlsx")

In [155]:
#Storing all the Power BI Apps data
app_id,app_name,workspaceId,lastUpdated,publishedBy=[],[],[],[],[]
appsUrl="https://api.powerbi.com/v1.0/myorg/apps"
appsUrl_response = requests.get(appsUrl,headers=headers)
if appsUrl_response.status_code == 200:
    jsonstr = appsUrl_response.content
    apps = json.loads(jsonstr)
    tempapps = apps['value'] 
    for i in range(len(tempapps)):
        app_id.append(tempapps[i]['id'])
        app_name.append(tempapps[i]['name'])
        workspaceId.append(tempapps[i]['workspaceId'])
        lastUpdated.append(tempapps[i]['lastUpdate'])
        publishedBy.append(tempapps[i]['publishedBy'])
else:
    Errors.append(["Error at App_Data",appsUrl_response])
Dict = {'AppID':app_id,'AppName':app_name,'WorkspaceId':workspaceId,'LastUpdated':lastUpdated,
        'PublishedBy':publishedBy}
df_appsData = pd.DataFrame(Dict)

In [156]:
Errors

[['Error at Dataset_UsersDetails in 3563efcb-ffe1-4369-b4bb-467d4d47c48d',
  <Response [401]>],
 ['Error at Dataset_UsersDetails in b07832c6-31c9-4153-825e-207685400d5f',
  <Response [401]>]]

In [157]:
#Storing all the Power BI Apps data
app_id,app_name,workspaceId,lastUpdated,publishedBy=[],[],[],[],[]
appsUrl = "https://api.powerbi.com/v1.0/myorg/apps"
appsUrl_response = requests.get(appsUrl,headers=headers)
if appsUrl_response.status_code == 200:
    jsonstr = appsUrl_response.content
    apps = json.loads(jsonstr)
    tempapps = apps['value'] 
    for i in range(len(tempapps)):
        app_id.append(tempapps[i]['id'])
        app_name.append(tempapps[i]['name'])
        workspaceId.append(tempapps[i]['workspaceId'])
        lastUpdated.append(tempapps[i]['lastUpdate'])
        publishedBy.append(tempapps[i]['publishedBy'])
else:
    Errors.append(["Error at App_Data",appsUrl_response])
Dict = {'AppID':app_id,'AppName':app_name,'WorkspaceId':workspaceId,'LastUpdated':lastUpdated,
        'PublishedBy':publishedBy}
df_appsData = pd.DataFrame(Dict)

In [158]:
Errors

[['Error at Dataset_UsersDetails in 3563efcb-ffe1-4369-b4bb-467d4d47c48d',
  <Response [401]>],
 ['Error at Dataset_UsersDetails in b07832c6-31c9-4153-825e-207685400d5f',
  <Response [401]>]]

In [159]:
#App data excel creation
df_appsData.to_excel("App_Data.xlsx")

In [160]:
#Storing all the Power BI Apps data
org_apps_url="https://api.powerbi.com/v1.0/myorg/apps"
org_apps_url_response = requests.get(org_apps_url,headers=headers)
#App Fields
appName = []
workspaceId = []
lastUpdated = []
publishedBy = []
#App Reports Fileds
reportID = []
reportType = []
reportName = []
webUrl = []
datasetId = []
appId = []
originalReportObjectId = []
users = []
subscriptions = []
sections = []
if org_apps_url_response.status_code == 200:
    jsonstr = org_apps_url_response.content
    apps = json.loads(jsonstr)
    appsData = apps['value']
    for app in appsData:
        appReportsUrl="https://api.powerbi.com/v1.0/myorg/apps/"+app['id']+"/reports"
        appReportsUrl_response = requests.get(appReportsUrl,headers=headers)
        if appReportsUrl_response.status_code == 200:
            appReportsJsonstr = appReportsUrl_response.content
            appReports = json.loads(appReportsJsonstr)
            appReportsData = appReports['value']
            for report in appReportsData:
                #App Fields
                appName.append(app['name'])
                workspaceId.append(app['workspaceId'])
                lastUpdated.append(app['lastUpdate'])
                publishedBy.append(app['publishedBy'])  
                #App Reports Fileds
                reportID.append(report['id'])
                reportType.append(report['reportType'])
                reportName.append(report['name'])
                webUrl.append(report['webUrl'])
                datasetId.append(report['datasetId'])
                appId.append(report['appId'])
                originalReportObjectId.append(report['originalReportObjectId'])
                users.append(len(report['users']))
                subscriptions.append(report['subscriptions'])
        else:
            Errors.append(["Error at AppReportsData at "+app['id'],appReportsUrl_response])
else:
    Errors.append(["Error at AppReportsData",org_apps_url_response])           
AppReportsDataDic = {'AppName':appName,'WorkspaceID':workspaceId,'LastUpdated':lastUpdated,'PublishedBy':publishedBy,'ReportID':reportID,
                     'ReportType':reportType,'ReportName':reportName,'ReportWebUrl':webUrl,'ReportDatasetID':datasetId,'AppID':appId,
                     'OriginalReportObjectID':originalReportObjectId,'UsersCount':users,'Subscriptions':subscriptions,}
df_AppReportsDataDic = pd.DataFrame(AppReportsDataDic)

In [161]:
#AppReports data excel creation
df_AppReportsDataDic.to_excel("AppReportsData.xlsx")

In [162]:
#Pulling all the reports from complete Org from My Workspace
Reports=[]
Report_Id = []
reportType = []
Report_Name = []
webUrl = []
Dataset_Id = []
appId = []
datasetWorkspaceId = []
myorg_reports_url="https://api.powerbi.com/v1.0/myorg/reports"
myorg_reports_response = requests.get(myorg_reports_url,headers=headers)
if myorg_reports_response.status_code == 200:
    jsonstr = myorg_reports_response.content
    Report = json.loads(jsonstr)
    tempReport = Report['value']
    
    for report in tempReport:
        Report_Id.append(report['id'])
        reportType.append(report['reportType'])
        Report_Name.append(report['name'])
        webUrl.append(report['webUrl'])
        Dataset_Id.append(report['datasetId'])
        try:
            appId.append(report['appId'])
        except:
            appId.append("NA")
        try:
            datasetWorkspaceId.append(report['datasetWorkspaceId'])
        except:
            datasetWorkspaceId.append("NA")
else:
    Errors.append(["Error at OrgReportsData",myorg_reports_response])
Org_Reports = {'ReportID':Report_Id,'ReportType':reportType,'ReportName':Report_Name,'WebUrl':webUrl,
               'DatasetID':Dataset_Id,'AppID':appId,'DatasetWorkspaceID':datasetWorkspaceId}
df_Org_Reports=pd.DataFrame(Org_Reports)

In [163]:
#All Org Reports Data
df_Org_Reports.to_excel("OrgReportsData.xlsx")

In [164]:
#Pulling all the reports present in a Workspace
ReportID = []
ReportName = []
weburl = []
DatasetID = []
DatasetWorkspaceID  = []
workspaceNames = []
weburl=[]
reportType = []
embedUrl = []
isFromPbix = []
for index, row in df_workspace_names_ids.iterrows():
    workspaceUrl = "https://api.powerbi.com/v1.0/myorg/groups/"+row['WorkspaceID']+"/reports"
    workspaceUrlResponse = requests.get(workspaceUrl,headers=headers)
    if workspaceUrlResponse.status_code == 200:
        workspaceUrlJsonstr = workspaceUrlResponse.content
        Reports = json.loads(workspaceUrlJsonstr)
        ReportsData = Reports['value']
        #print(ReportsData)
        
        for report in ReportsData:
            #print(report)
            ReportID.append(report['id'])
            ReportName.append(report['name'])
            weburl.append(report['webUrl'])
            isFromPbix.append(report['isFromPbix'])
            embedUrl.append(report['embedUrl'])
            reportType.append(report['reportType'])
            workspaceNames.append(row['WorkspaceName'])
            if 'datasetId' in report:
                DatasetID.append(report['datasetId'])
            else:
                DatasetID.append("No Dataset")
            if 'datasetWorkspaceId' in report:
                DatasetWorkspaceID.append(report['datasetWorkspaceId'])
            else:
                DatasetWorkspaceID.append("No datasetWorkspaceId")
    else:
        Errors.append(["workspaceUrlJsonstr",myorg_reports_response])

tempReports = {'WorkspaceName':workspaceNames,'ReportName':ReportName,'ReportID':ReportID,
               'DatasetID':DatasetID,'DatasetWorkspaceID':DatasetWorkspaceID, 'ReportType':reportType,
                   'EmbedUrl':embedUrl,'IsFromPbix':isFromPbix,'WebUrl':weburl}
df_Reports = pd.DataFrame(tempReports)

In [ ]:
Errors

In [165]:
#All Org Reports Data
df_Reports.to_excel("ReportsDataBasedOnWorkspace.xlsx")

In [166]:
#Pulling Page of a Report
ReportID = []
ReportSection = []
PageName = []
PageOrder = []

for index, row in df_Reports.iterrows():
    reportPageUrl =  "https://api.powerbi.com/v1.0/myorg/reports/"+row['ReportID']+"/pages"
    reportPageResponse = requests.get(reportPageUrl,headers=headers)
    if reportPageResponse.status_code == 200:
        reportPageJsonstr = reportPageResponse.content
        ReportPages = json.loads(reportPageJsonstr)
        ReportPagesData = ReportPages['value']
        #print(ReportsData)
        
        for page in ReportPagesData:
            ReportID.append(row['ReportID'])
            if 'name' in page:
                ReportSection.append(page['name'])
            else:
                ReportSection.append("NA")
            if 'displayName' in page:
                PageName.append(page['displayName'])
            else:
                PageName.append("NA")
            if 'order' in page:
                PageOrder.append(page['order'])
            else:
                PageOrder.append('NA')
    else:
        Errors.append(["workspaceUrlJsonstr",myorg_reports_response])

tempReportPages = {'ReportID':ReportID,'ReportSection':ReportSection,'PageName':PageName,
               'PageOrder':PageOrder}
df_ReportPages = pd.DataFrame(tempReportPages)

In [ ]:
Errors

In [167]:
#All Org Reports Data
df_ReportPages.to_excel("ReportPages.xlsx")

In [168]:
#Pulling all Refresh history of Dataset

RequestID = []
ID = []
RefreshType = []
StartTime = []
EndTime = []
Status = []
RefreshAttempts = []
DatasetId = []

for index, row in df_Workspace_Datasets.iterrows():
    refreshHistoryURL = "https://api.powerbi.com/v1.0/myorg/groups/"+row['WorkspaceID']+"/datasets/"+row['DatasetID']+"/refreshes"
    refreshHistoryResponse = requests.get(refreshHistoryURL,headers=headers)
    if refreshHistoryResponse.status_code == 200:
        refreshHistoryJsonstr = refreshHistoryResponse.content
        refreshHistory = json.loads(refreshHistoryJsonstr)
        refreshHistoryData = refreshHistory['value']
        # print(refreshHistoryData)
        if(len(refreshHistoryData)!=0):
            # print("Yes")
            for refreshData in refreshHistoryData:
                #print(report)
                RequestID.append(refreshData['requestId'])
                ID.append(refreshData['id'])
                RefreshType.append(refreshData['refreshType'])
                StartTime.append(refreshData['startTime'])
                EndTime.append(refreshData['endTime'])
                Status.append(refreshData['status'])
                RefreshAttempts.append(refreshData['refreshAttempts'])
                DatasetId.append(row['DatasetID'])
        else:
            RequestID.append("NA")
            ID.append("NA")
            RefreshType.append("NA")
            StartTime.append("NA")
            EndTime.append("NA")
            Status.append("NA")
            RefreshAttempts.append("NA")
            DatasetId.append(row['DatasetID'])
    else:
        Errors.append(["DatasetRefreshHistory in", row['DatasetID']+ " " + row['DatasetName'],refreshHistoryResponse])

tempRefreshHistroy = {'RequestID':RequestID,'ID':ID,'RefreshType': RefreshType,'StartTime':StartTime,
                      'EndTime':EndTime, 'Status':Status, 'RefreshAttempts':RefreshAttempts,'DatasetID':DatasetId}
df_RefreshHistroy = pd.DataFrame(tempRefreshHistroy)

In [169]:
Errors

[['Error at Dataset_UsersDetails in 3563efcb-ffe1-4369-b4bb-467d4d47c48d',
  <Response [401]>],
 ['Error at Dataset_UsersDetails in b07832c6-31c9-4153-825e-207685400d5f',
  <Response [401]>]]

In [170]:
#Dataset Refresh Histroy Excel
df_RefreshHistroy.to_excel("DatasetRefreshHistory.xlsx")

In [171]:
#Stroing All Dashboard details based on Workspace IDs
WorkspaceName = []
DashboardName = []
DashboardId = []
IsReadOnly = []
WebUrl = []
df_Workspace_Report_Dataset = pd.DataFrame(Dict)
for index, row in df_workspace_names_ids.iterrows():
    dashboardUrl = "https://api.powerbi.com/v1.0/myorg/groups/"+row['WorkspaceID']+"/dashboards"
    dashboardResponse = requests.get(dashboardUrl,headers=headers)
    if dashboardResponse.status_code == 200:
        dashboardJsonstr = dashboardResponse.content
        dashboard = json.loads(dashboardJsonstr)
        dashboardData = dashboard['value']
        if(len(dashboardData)==0):
            WorkspaceName.append(row['WorkspaceName'])
            DashboardId.append("NA")
            DashboardName.append("NA")
            IsReadOnly.append("NA")
            WebUrl.append("NA")
        else:
            WorkspaceName.append(workspaceNames[Id])
            DashboardId.append(dashboardData[0]['id'])
            DashboardName.append(dashboardData[0]['displayName'])
            IsReadOnly.append(dashboardData[0]['isReadOnly'])
            WebUrl.append(dashboardData[0]['webUrl'])  
    else:
        Errors.append("Error at dashboards Data")
DashboardsDic = {'WorkspaceName':WorkspaceName,'DashboardName':DashboardName,'DashboardID':DashboardId,'IsReadOnly':IsReadOnly,'WebUrl':WebUrl}
Dashboards_df = pd.DataFrame(DashboardsDic)

In [172]:
#Dashboards data excel creation
Dashboards_df.to_excel("Dashboards_Data.xlsx")

In [173]:
#Pulling all the locations from GIT everytime we need to create clone 
folder_path = "C://Users//AkhilGrandhi//Downloads//PBIADO/Testing"
bim_paths = []
pbir_paths = []
target_extensions = ['.bim', '.pbir']
# Use os.walk to traverse the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        # print(file)
        file_path = os.path.join(root, file)
        # Check if the file has a .bim extension
        if file.endswith('.bim'):
            bim_paths.append(file_path)
        # Check if the file has a .pbir extension
        elif file.endswith('.pbir'):
            pbir_paths.append(file_path)
# x,y=[],[]
#print(len(pbir_paths))
# for i in pbir_paths:
#     if "\\Reports" in i:
#         pass
#     else:
#         x.append(i)
# for i in bim_paths:
#     if "\\Reports" not in i:
#         y.append(i)
# #print(len(x),len(y))
# bim_paths,pbir_paths = y,x

In [174]:
# Roles Data

# MQuery = []
DatasetNames = []
WorkspaceNames = []
RoleName = []
ModelPermission = []
TableName = []
FilterExpression = []


for i in range(len(bim_paths)):
    
    bim_file = bim_paths[i]

    if(bim_paths[i].endswith('.bim') and bim_paths[i].split('\\')[-2].endswith('.Dataset')):
        DatasetName = bim_paths[i].split('\\')[-2][:-8]
    elif(bim_paths[i].endswith('.bim') and bim_paths[i].split('\\')[-2].endswith('.SemanticModel')):
        DatasetName = bim_paths[i].split('\\')[-2][:-14]
    WorkspaceName = bim_paths[i].split('\\')[-3]
    # print(bim_file)
    # print("---->",DatasetName)
    with open(bim_file, 'r', encoding='utf-8') as bim:
        bim_data = json.load(bim)

    if('roles' in bim_data['model']):
        for role in bim_data['model']['roles']:
            
            if('tablePermissions' in role):
                for TE in role['tablePermissions']:
                    DatasetNames.append(DatasetName)
                    WorkspaceNames.append(WorkspaceName)
                    RoleName.append(role['name'])
                    ModelPermission.append(role['modelPermission'])
                    TableName.append(TE['name'])
                    FilterExpression.append(TE['filterExpression'])
                else:
                    DatasetNames.append(DatasetName)
                    WorkspaceNames.append(WorkspaceName)
                    RoleName.append(role['name'])
                    ModelPermission.append(role['modelPermission'])
                    TableName.append('NA')
                    FilterExpression.append('NA')
    else:
        DatasetNames.append(DatasetName)
        WorkspaceNames.append(WorkspaceName)
        RoleName.append("NA")
        ModelPermission.append('NA')
        TableName.append('NA')
        FilterExpression.append('NA')

RoleDataDic = {'DatasetName': DatasetNames, 'WorkspaceName': WorkspaceNames, 'RoleName': RoleName, 
               'ModelPermission': ModelPermission, 'TableName': TableName, 'FilterExpression': FilterExpression}

df_RoleDataDic = pd.DataFrame(RoleDataDic)
            
        

In [175]:
#Roles data excel creation
df_RoleDataDic.to_excel("Roles.xlsx")

In [176]:
# Tables Data

# MQuery = []
TableNames = []
TableType = []
ColumnNames = []
ColumnType = []
ColumnExpression = []
DataTypes = []
SourceProviderType = []
MeasureNames = []
DaxExpressions = []
DatasetNames = []
WorkspaceNames = []


# ADO_WorkspaceNames = ['Cloud','Cloud BA','CommissionAnalytics','CSOps',
#                       'Finance', 'FY-Frozen','GenAI','GlobalOps',
#                       'HR','Keystone','NLS','PBI Utility','PS','Sales','T&E']

# Org_WorkSpaceNames = [
#     'Cloud_Adhoc_Model_Prod','Cloud BA_Adhoc_Model_Prod','CommissionAnalytics_Model_Prod','CSOPS_Adhoc_Model_Prod',
#     'Finance_Adhoc_Model_Prod','FY_Frozen_Adhoc_Model_Prod','NA','GlobalOps_Adhoc_Model_Prod','HR_Adhoc_Model_Prod',
#     'Keystone_Adhoc_Model_Prod','Nls_Adhoc_Model_Prod',
#     'Microsoft Fabric Capacity Metrics EDA','PS_Adhoc_Model_Prod',
#     'Sales_Adhoc_Model_Prod','T&E-Adhoc_Model_Prod'
#     ]

for i in range(len(bim_paths)):
    
    bim_file = bim_paths[i]
    #print(bim_paths[i].split('\\'))
    if(bim_paths[i].endswith('.bim') and bim_paths[i].split('\\')[-2].endswith('.Dataset')):
        DatasetName = bim_paths[i].split('\\')[-2][:-8]
    elif(bim_paths[i].endswith('.bim') and bim_paths[i].split('\\')[-2].endswith('.SemanticModel')):
        DatasetName = bim_paths[i].split('\\')[-2][:-14]
    #WorkspaceName = Org_WorkSpaceNames[ADO_WorkspaceNames.index(bim_paths[i].split('\\')[-4])]
    WorkspaceName = bim_paths[i].split('\\')[-3]
    # print(bim_file)
    # print("---->",DatasetName)
    with open(bim_file, 'r', encoding='utf-8') as bim:
        bim_data = json.load(bim)
    if('tables' in bim_data['model']):
        for table in bim_data['model']['tables']:
            # print("Table--->",table['name'])
            if('columns' in table):
                maxlen = len(table['columns'])
                for column in table['columns']:
                    ColumnNames.append(column['name'])
                    if('dataType' in column):
                        DataTypes.append(column['dataType'])
                    else:
                        DataTypes.append('NA')
                    if('type' in column):
                        ColumnType.append(column['type'])
                    else:
                        ColumnType.append('Column')
                    if('expression' in column):
                        CC_Exp = ''
                        for cc_ex in column['expression']:
                            CC_Exp += cc_ex
                        ColumnExpression.append(CC_Exp)
                    else:
                        ColumnExpression.append('NA')
                    if('sourceProviderType' in column):
                        SourceProviderType.append(column['sourceProviderType'])
                    else:
                        SourceProviderType.append('NA')
                    TableNames.append(table['name'])
                    if('mode' in table['partitions'][0]):
                        TableType.append(table['partitions'][0]['mode'])
                    else:
                        TableType.append('NA')
                    DatasetNames.append(DatasetName)
                    WorkspaceNames.append(WorkspaceName)
            if('measures' in table):
                if('columns' in table):
                    maxlen = max(len(table['columns']),len(table['measures']))
                else:
                    maxlen = (len(table['measures']))
                for measure in table['measures']:
                    MeasureNames.append(measure['name'])
                    if('expression' in measure):
                        daxExp = ''
                        for ex in measure['expression']:
                            daxExp += ex
                        DaxExpressions.append(daxExp)
                    else:
                        DaxExpressions.append('NA')
                
                if('columns' in table):
                    if(len(table['columns'])>len(table['measures'])):
                        diff = len(table['columns'])-len(table['measures']) 
                        MeasureNames += ['NA']*diff
                        DaxExpressions += ['NA']*diff

                    elif(len(table['columns'])<len(table['measures'])):
                        diff = len(table['measures'])-len(table['columns'])
                        TableNames += [table['name']]*diff
                        if('mode' in table['partitions'][0]):
                            TableType += [table['partitions'][0]['mode']]*diff
                        else:
                            TableType += ['NA']*diff
                        ColumnNames += ['NA']*diff
                        ColumnType += ['NA']*diff
                        ColumnExpression += ['NA']*diff
                        DataTypes += ['NA']*diff
                        SourceProviderType += ['NA']*diff
                        DatasetNames += [DatasetName]*diff
                        WorkspaceNames += [WorkspaceName]*diff
                        
                    elif(len(table['columns'])==len(table['measures'])):
                        continue 
                if('measures' in table and 'columns' not in table):
                    TableNames += [table['name']]*maxlen
                    if('mode' in table['partitions'][0]):
                        TableType += [table['partitions'][0]['mode']]*maxlen
                    else:
                        TableType += ['NA']*maxlen
                    ColumnNames += ['NA']*maxlen
                    ColumnType += ['NA']*maxlen
                    ColumnExpression += ['NA']*maxlen
                    DataTypes += ['NA']*maxlen
                    SourceProviderType += ['NA']*maxlen
                    DatasetNames += [DatasetName]*maxlen
                    WorkspaceNames += [WorkspaceName]*maxlen
            else:
                MeasureNames += ['NA']*maxlen
                DaxExpressions += ['NA']*maxlen
    else:
        TableNames += ["No Tables"]
        DatasetNames += [DatasetName]
        WorkspaceNames += [WorkspaceName]
        TableType += ['NA']
        ColumnNames += ['NA']
        ColumnType += ['NA']
        ColumnExpression += ['NA']
        DataTypes += ['NA']
        SourceProviderType += ['NA']
             
        
                       
data_dict = {
'TableName': TableNames,
'TableType': TableType,
'ColumnName': ColumnNames,
'ColumnType': ColumnType,
'ColumnExpression': ColumnExpression,
'DataTypes': DataTypes,
'SourceProviderType': SourceProviderType,
'MeasureName': MeasureNames,
'DaxExpression': DaxExpressions,
'DatasetName': DatasetNames,
'WorkspaceName': WorkspaceNames 
}
df_tablesData = pd.DataFrame.from_dict(data_dict)      

In [177]:
df_tablesData.to_excel("ModelMetaData.xlsx")

In [178]:
#Relationships Data
relationship_data=[]
fromColumn = []
fromTable = []
toColumn = []
toTable = []
mainDatasetList = []
mainWorkspaceList = []
crossFilteringBehavior = []
toCardinality = []
securityFilteringBehavior = []
for i in range(len(bim_paths)):
    bim_file = bim_paths[i]
    #print(bim_paths[i].split('\\'))
    if(bim_paths[i].endswith('.bim') and bim_paths[i].split('\\')[-2].endswith('.Dataset')):
        DatasetName = bim_paths[i].split('\\')[-2][:-8]
    elif(bim_paths[i].endswith('.bim') and bim_paths[i].split('\\')[-2].endswith('.SemanticModel')):
        DatasetName = bim_paths[i].split('\\')[-2][:-14]
    WorkspaceName = bim_paths[i].split('\\')[-3]
    try:
        for relationship in bim_data['model']['relationships']:
            mainWorkspaceList.append(WorkspaceName)
            mainDatasetList.append(DatasetName)
            fromColumn.append(relationship['fromColumn'])
            fromTable.append(relationship['fromTable'])
            toColumn.append(relationship['toColumn'])
            toTable.append(relationship['toTable'])
            if('crossFilteringBehavior' in relationship):
                crossFilteringBehavior.append(relationship['crossFilteringBehavior'])
            else:
                crossFilteringBehavior.append('oneDirection')
            if('toCardinality' in relationship):
                toCardinality.append(relationship['toCardinality'])
            else:
                toCardinality.append('one')
            if('securityFilteringBehavior' in relationship):
                securityFilteringBehavior.append(relationship['securityFilteringBehavior'])
            else:
                securityFilteringBehavior.append('NA')
    except Exception as error:
        # print(DatasetName,error)
        # print(relationship)
        mainWorkspaceList.append(WorkspaceName)
        mainDatasetList.append(DatasetName)
        fromColumn.append("NA")
        fromTable.append("NA")
        toColumn.append("NA")
        toTable.append("NA")
        crossFilteringBehavior.append("NA")
        toCardinality.append("NA")
        
data_dict = {
        'Workspacename': mainWorkspaceList,
        'DatasetName': mainDatasetList, 
        'From Column': fromColumn, 
        'From Table': fromTable,
        'To Column': toColumn,
        'To Table': toTable,
        'CrossFilteringBehavior': crossFilteringBehavior,
        'ToCardinality': toCardinality,
        'SecurityFilteringBehavior': securityFilteringBehavior
    }

df_relationshipsdata = pd.DataFrame.from_dict(data_dict)      

In [179]:
#Relationships data excel creation
df_relationshipsdata.to_excel("RelationshipsData.xlsx")

In [ ]:
Identifier = []
UserPrincipalName = []
DisplayName = []

#Azure App Details
TENANT_ID = ''
CLIENT_ID = ''
CLIENT_SECRET = ''
GROUP_ID = '6c287664-023b-4dcc-aa32-6532818d2fa3'  # Your group Object ID
url = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

Azureheaders = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "scope": "https://graph.microsoft.com/.default"
}

IdentifierResponse = requests.post(url, headers=Azureheaders, data=data)
IdentifierResponse.raise_for_status()  # Raise an error for bad status
# print(response.json().get("access_token"))
access_token = IdentifierResponse.json().get("access_token")

uniqueGroupIdentifiers = list(set(df_usersDetails[df_usersDetails['PrincipalType'] == 'Group']['Identifier']))

for identifier in uniqueGroupIdentifiers:
    # print(identifier)
    groupMembersUrl = "https://graph.microsoft.com/v1.0/groups/"+identifier+"/members"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    # print(groupMembersResponse)
    groupMembersResponse = requests.get(groupMembersUrl, headers=headers)
    if(groupMembersResponse.status_code == 200):
        members = groupMembersResponse.json().get("value", [])
    
    for member in members:
        Identifier.append(identifier)
        UserPrincipalName.append(member['userPrincipalName'])
        DisplayName.append(member['displayName'])


groupMembersDict = {'Identifier': Identifier, 'UserPrincipalName': UserPrincipalName, 'DisplayName': DisplayName}

df_groupMembers = pd.DataFrame(groupMembersDict)

In [185]:
#Datasets user details excel creation
df_groupMembers.to_excel("IdentifierMembers.xlsx")

In [186]:
Identifier = []
IdentifierName = []
CreatedDateTime = []
Mail = []
for identifier in uniqueGroupIdentifiers:
    groupDetailsUrl = "https://graph.microsoft.com/v1.0/groups/"+identifier
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    groupDetailsResponse = requests.get(groupDetailsUrl, headers=headers)
    if(groupDetailsResponse.status_code == 200):
        groupJsonstr = groupDetailsResponse.content
        groupData = json.loads(groupJsonstr)
        Identifier.append(groupData['id'])
        IdentifierName.append(groupData['displayName'])
        CreatedDateTime.append(groupData['createdDateTime'])
        Mail.append(groupData['mail'])

groupDataDic = {'Identifier': Identifier, 'IdentifierName': IdentifierName, 'CreatedDateTime': CreatedDateTime, 'Mail': Mail}
df_groupData = pd.DataFrame(groupDataDic)
        
    

In [187]:
#Datasets user details excel creation
df_groupData.to_excel("IdentifierDetails.xlsx")